In [ ]:
from osgeo import gdal
from datetime import date, datetime as dt
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("nbagg")
import matplotlib.pyplot as plt

## Read Planet Data

In [ ]:
planet_fn = "/data/001_planet_sentinel_study/planet/utm11n_sur_ref/ndvi/planet_ndvi.vrt"
planet = gdal.Open(planet_fn)


planet_fns = planet.GetFileList()[1:]
planet_dates = [dt.strptime((f.split("/"))[-1][0:15], "%Y%m%d_%H%M%S") for f in planet_fns]

planet_ndvi = planet.ReadAsArray()
print planet_ndvi.shape

## Read Sentinel Data

In [ ]:
sentinel_fn = "/data/001_planet_sentinel_study/sentinel/11SKA_NDVI/sen2_boa_ndvi.vrt"
sentinel = gdal.Open(sentinel_fn)

sentinel_fns = sentinel.GetFileList()[1:]
sentinel_dates = [dt.strptime((f.split("/"))[-1][5:20], "%Y%m%d_%H%M%S") for f in sentinel_fns]

sentinel_ndvi = sentinel.ReadAsArray()
print sentinel_ndvi.shape

# Get lat/lon information
transf = sentinel.GetGeoTransform()
reslon = transf[1]
reslat = transf[5]
cols = sentinel.RasterXSize
rows = sentinel.RasterYSize
LHS = transf[0]
THS = transf[3]
BHS = THS + (rows * reslat)
RHS = LHS + (cols * reslon)

latgrid = np.arange(THS, BHS, reslat)
longrid = np.arange(LHS, RHS, reslon)

## Extract Values from NDVI stack

In [ ]:
tl_pixel_lat = 4062320
tl_pixel_lon = 235443

br_pixel_lat = 4061520
br_pixel_lon = 236117

# ===== GET ROW/COLS FOR PIXEL LAT/LON ===== #

tl_pix_x = abs(longrid - tl_pixel_lon).argmin()
tl_pix_y = abs(latgrid - tl_pixel_lat).argmin()
br_pix_x = abs(longrid - br_pixel_lon).argmin()
br_pix_y = abs(latgrid - br_pixel_lat).argmin()

# ===== EXTRACT SENTINEL VALUES ===== #

sentinel_values = []

for i in range(0, (sentinel_ndvi.shape)[0]):
        
    sentinel_values.append(np.nanmean(sentinel_ndvi[i, tl_pix_y:br_pix_y, tl_pix_x:br_pix_x]))
    
sentinel_df = pd.DataFrame(index = sentinel_dates)
sentinel_df['ndvi'] = sentinel_values

# ===== EXTRACT PLANET VALUES ===== #

planet_values = []

for i in range(0, (planet_ndvi.shape)[0]):
        
    planet_values.append(np.nanmean(planet_ndvi[i, tl_pix_y:br_pix_y, tl_pix_x:br_pix_x]))

planet_df = pd.DataFrame(index = planet_dates)
planet_df['ndvi'] = planet_values

# Get rid of NaN values present in a row
#planet_df = planet_df[np.isfinite(planet_df['ndvi'])]

print "Done."

## Extract Values from MODIS csv file

In [ ]:
# ===== MCD43 OPENING... ===== #

MCD43 = pd.DataFrame.from_csv("/media/DataShare/Taylor/modis/MCD43/bdrf-235799-4061930/BDRF-235799-4061930-MCD43A1-006-results.csv", index_col=2)

MCD43_df = pd.DataFrame(index = MCD43.index)
MCD43_df['nir'] = MCD43['MCD43A1_006_BRDF_Albedo_Parameters_Band2_0']
MCD43_df['red'] = MCD43['MCD43A1_006_BRDF_Albedo_Parameters_Band1_0']
MCD43 = None

#MCD43_df = MCD43_df[MCD43_df.nir < 0.4]

MCD43_df['ndvi'] = ((MCD43_df.nir-MCD43_df.red)/(MCD43_df.nir+MCD43_df.red))

# Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ax.plot_date(planet_df.index, planet_df['ndvi'], '-', color='Blue', linewidth=2)

ax.plot_date(MCD43_df.index, MCD43_df['ndvi'], '-', color='Black', linewidth=2)

ax.plot_date(sentinel_df.index, sentinel_df['ndvi'], '-', color='red', linewidth=2)

fig.autofmt_xdate()
ax.set_xlim([date(2017, 3, 1), date(2017, 9, 30)])
ax.set_ylim([0, 0.7])
plt.show()

In [ ]:
plt.imshow(planet_ndvi[11,:,:], extent=[LHS, RHS, BHS, THS])
plt.show()